In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import os
from six.moves.urllib.request import urlopen

In [3]:
import tensorflow as tf
import numpy as np

In [4]:
IRIS_TRAINING = 'iris_train.csv'
IRIS_TRAINING_URL = 'http://download.tensorflow.org/data/iris_training.csv'

IRIS_TEST = 'iris_test.csv'
IRIS_TEST_URL = 'http://download.tensorflow.org/data/iris_test.csv'

In [5]:
#下载数据集
if not os.path.exists(IRIS_TRAINING):
    raw = urlopen(IRIS_TRAINING_URL).read()
    with open(IRIS_TRAINING , 'wb') as f:
        f.write(raw)

if not os.path.exists(IRIS_TEST):
    raw = urlopen(IRIS_TEST_URL).read()
    with open(IRIS_TEST , 'wb') as f:
        f.write(raw)

In [6]:
#从csv中读入数据
#pandas也可以来处理csv 且更加强大

#target成员和data成员（feature）

#Datasets in tf.contrib.learn are named tuples; 
#with_header
#without_header
#上面区别 就是csv文件最上面是否有列描述
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(filename=IRIS_TRAINING,
                                                                  target_dtype = np.int,
                                                                  features_dtype = np.float32)

test_set = tf.contrib.learn.datasets.base.load_csv_with_header(filename=IRIS_TEST,
                                                              target_dtype = np.int,
                                                              features_dtype = np.float32)

In [9]:
training_set.data#feature
training_set.target

array([2, 1, 2, 0, 0, 0, 0, 2, 1, 0, 1, 1, 0, 0, 2, 1, 2, 2, 2, 0, 2, 2, 0,
       2, 2, 0, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 0, 0, 2,
       0, 2, 0, 2, 0, 1, 1, 0, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 0, 2, 2,
       0, 0, 1, 0, 2, 2, 0, 1, 1, 1, 2, 0, 1, 1, 1, 2, 0, 1, 1, 1, 0, 2, 1,
       0, 0, 2, 0, 0, 2, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 2, 1, 0, 2, 0,
       1, 1, 0, 0, 1])

In [12]:
#直接使用已经实现好了的model
#指定所有特性都具有实值数据
#因为只有4个feature所以shape=4
feature_columns = [tf.feature_column.numeric_column('x' , shape=[4])]
#类似一个占位符

#创建一个dnn
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                       hidden_units = [10,20,10],
                                       n_classes = 3,
                                       model_dir = 'iris_model')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'iris_model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [14]:
#创建训练数据
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x':np.array(training_set.data)},
                                                   y = np.array(training_set.target),
                                                   num_epochs = None,
                                                   shuffle = True)

In [15]:
#接下来开始训练模型
classifier.train(input_fn=train_input_fn , steps=2000)

#classifier 会保存模型的状态 所以可以将训练步骤随便拆开
#下面两次1000就相当于上面的一次2000
classifier.train(input_fn=train_input_fn , steps=1000)
classifier.train(input_fn=train_input_fn , steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into iris_model\model.ckpt.
INFO:tensorflow:loss = 278.491, step = 1
INFO:tensorflow:global_step/sec: 598.375
INFO:tensorflow:loss = 9.75565, step = 101 (0.169 sec)
INFO:tensorflow:global_step/sec: 566.17
INFO:tensorflow:loss = 10.6328, step = 201 (0.176 sec)
INFO:tensorflow:global_step/sec: 561.402
INFO:tensorflow:loss = 8.78367, step = 301 (0.180 sec)
INFO:tensorflow:global_step/sec: 511.146
INFO:tensorflow:loss = 8.49244, step = 401 (0.195 sec)
INFO:tensorflow:global_step/sec: 427.961
INFO:tensorflow:loss = 12.3439, step = 501 (0.238 sec)
INFO:tensorflow:global_step/sec: 333.098
INFO:tensorflow:loss = 13.2094, step = 601 (0.297 sec)
INFO:tensorflow:global_step/sec: 511.144
INFO:tensorflow:loss = 6.79398, step = 701 (0.201 sec)
INFO:tensorflow:global_step/sec: 412.079
INFO:tensorflow:loss = 8.88533, step = 801 (0.236 sec)
INFO:tensorflow:global_step/sec: 591.298
INFO:tensorflow:loss = 2.58342, step =

In [20]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x':np.array(test_set.data)},
                                                  y = np.array(test_set.target),
                                                  num_epochs = 1,
                                                  shuffle = False)
accuracy = classifier.evaluate(input_fn=test_input_fn)['accuracy']

INFO:tensorflow:Starting evaluation at 2017-11-07-02:38:38
INFO:tensorflow:Restoring parameters from iris_model\model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2017-11-07-02:38:38
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.966667, average_loss = 0.0657732, global_step = 2000, loss = 1.9732


In [21]:
print('test accuracy:' , accuracy)

test accuracy: 0.966667


In [36]:
#实际运行几个例子尝试一下
new_samples = np.array([
    [6.4,3.2,4.5,1.5],
    [5.8,3.1,5.0,1.7]] , dtype = np.float32)

predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x':new_samples},
                                                     num_epochs = 1,
                                                     shuffle = False)

prediction = list(classifier.predict(input_fn=predict_input_fn))


INFO:tensorflow:Restoring parameters from iris_model\model.ckpt-2000


In [43]:
print(int(prediction[0]['classes'][0]))
print(int(prediction[1]['classes'][0]))

1
2
